In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

# Import environment variables
start_date = os.getenv('start_date')
end_date = os.getenv('end_date')

In [54]:
cwd = os.getcwd()  # get current working directory

# Get the parent directory of the current working directory
grandparent_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))

In [55]:
btc_dominance = pd.read_csv(os.path.join(grandparent_dir, "data\\raw\\files\\bitcoin-dominance_(Coinmarketcap).csv"))
data_gpr = pd.read_csv(os.path.join(grandparent_dir, "data\\raw\\files\\data_gpr_export.csv"))
market_cap = pd.read_csv(os.path.join(grandparent_dir, "data\\raw\\files\\market-cap_20130429_20240413_(Coinmarketcap).csv"))
market_volume = pd.read_csv(os.path.join(grandparent_dir, "data\\raw\\files\\volume-24h_20130429_20240413_(Coinmarketcap).csv"))
wtui_df = pd.read_csv(os.path.join(grandparent_dir, "data\\raw\\files\\WTUI_Data.csv"))
wui_df = pd.read_csv(os.path.join(grandparent_dir, "data\\raw\\files\\WUI_Data.csv"))

## Data Processing

In [56]:
btc_dominance['DateTime'] = pd.to_datetime(btc_dominance['DateTime']).dt.strftime("%Y-%m-%d")
btc_dominance = btc_dominance[btc_dominance['DateTime'] >= start_date].reset_index(drop=True)
btc_dominance.rename(columns={'DateTime': 'Date'}, inplace=True)

df_dominance = pd.DataFrame(btc_dominance)

# Select only numeric columns for summation
numeric_cols = df_dominance.select_dtypes(include=[float, int]).columns

# Calculate sum of numeric columns for each row
df_dominance['sum'] = df_dominance[numeric_cols].sum(axis=1)

# Calculate percentage for each column
for col in df_dominance.columns[1:-1]:  # Exclude 'Date', 'Sum', and 'Others'
    df_dominance[col + '_percentage_dominance'] = df_dominance[col] / df_dominance['sum']
    
df_dominance.columns = df_dominance.columns.str.upper()

df_dominance.rename(columns={'DATE': 'Date'}, inplace=True)

# Select specific columns to keep
specific_columns = ['BTC_PERCENTAGE_DOMINANCE', 'ETH_PERCENTAGE_DOMINANCE', 'USDT_PERCENTAGE_DOMINANCE', 'BNB_PERCENTAGE_DOMINANCE', 'SOL_PERCENTAGE_DOMINANCE', 'OTHERS_PERCENTAGE_DOMINANCE']

# Create a new DataFrame containing only the specific columns
df_dominance_edit = df_dominance[['Date'] + specific_columns].copy()

In [57]:
data_gpr['month'] = pd.to_datetime(data_gpr['month']).dt.strftime("%Y-%m-%d")
data_gpr.rename(columns={'month': 'Date'}, inplace=True)

In [58]:
market_cap['DateTime'] = pd.to_datetime(market_cap['DateTime']).dt.strftime("%Y-%m-%d")
market_cap.rename(columns={'DateTime': 'Date'}, inplace=True)

In [59]:
market_volume['DateTime'] = pd.to_datetime(market_volume['DateTime']).dt.strftime("%Y-%m-%d")
market_volume.rename(columns={'DateTime': 'Date'}, inplace=True)

In [60]:
# Convert 'year' to PeriodIndex with frequency 'Q'
wtui_df['year'] = pd.PeriodIndex(wtui_df['year'], freq='Q')
wtui_df['year'] = wtui_df['year'].astype('datetime64[ns]')

# Convert dates to the desired format
wtui_df['year'] = wtui_df['year'].dt.strftime('%Y-%m-%d')
wtui_df.rename(columns={'year': 'Date'}, inplace=True)

In [61]:
# Convert 'year' to PeriodIndex with frequency 'Q'
wui_df['year'] = pd.PeriodIndex(wui_df['year'], freq='Q')
wui_df['year'] = wui_df['year'].astype('datetime64[ns]')

# Convert dates to the desired format
wui_df['year'] = wui_df['year'].dt.strftime('%Y-%m-%d')
wui_df.rename(columns={'year': 'Date'}, inplace=True)

## Concatenate all datasets

In [62]:
# Create a dummy DataFrame with the date range
dummy_df = pd.DataFrame({'Date': pd.date_range(start=start_date, end=end_date)})
dummy_df['Date'] = pd.to_datetime(dummy_df['Date']).dt.strftime("%Y-%m-%d")

merged_df = pd.merge(dummy_df, df_dominance_edit, on='Date', how='left')
merged_df = pd.merge(merged_df, data_gpr, on='Date', how='left')
merged_df = pd.merge(merged_df, market_cap, on='Date', how='left')
merged_df = pd.merge(merged_df, market_volume, on='Date', how='left')
merged_df = pd.merge(merged_df, wtui_df, on='Date', how='left')
merged_df = pd.merge(merged_df, wui_df, on='Date', how='left')

In [ ]:
merged_df.to_parquet(os.path.join(grandparent_dir, "data\\processed\\files\\merged_df.parquet"), index=False)